In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import json
import numpy as np
import time
import random
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
topics = [
    'https://www.gamerbraves.com/category/news/',
    'https://www.gamerbraves.com/category/reviews/',
    'https://www.gamerbraves.com/category/interview/'
]

In [ ]:
r = requests.get(topics[0])
soup = BeautifulSoup(r.content, "lxml")

In [ ]:
nums = []
a = soup.find_all('a', {'class': 'page_number'})
for a_ in a:
    try:
        nums.append(int(a_.text.replace(',','')))
    except:
        pass
    
if len(nums):
    page_num = max(nums)
else:
    page_num = 1
    
page_num

In [ ]:
h3 = soup.find_all('h3', {'class': 'jeg_post_title'})
a = [h3_.find('a').get('href') for h3_ in h3]
a

In [11]:
!rm -rf url
!mkdir url

In [12]:
def crawl(url, filename):
    if os.path.exists(filename):
        return

    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    h3 = soup.find_all('h3', {'class': 'jeg_post_title'})
    a = [h3_.find('a').get('href') for h3_ in h3]

    with open(filename, 'w') as fopen:
        json.dump(a, fopen)

In [13]:
max_worker = 10

for t in topics:
    print(t)
    t_ = t.split('com/')[1].replace('/', '-')
    
    r = requests.get(t)
    soup = BeautifulSoup(r.content, "lxml")
    
    nums = []
    a = soup.find_all('a', {'class': 'page_number'})
    for a_ in a:
        try:
            nums.append(int(a_.text.replace(',','')))
        except:
            pass

    if len(nums):
        page_num = max(nums)
    else:
        page_num = 1
        
    for i in tqdm(range(1, page_num + 1, max_worker)):
        a = np.arange(i, i + max_worker)
        urls = [(f'{t}page/{k}', os.path.join('url', f'{t_}-{k}.json')) for k in a]
        
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url[0], url[1]): url for url in urls}
            
            for future in as_completed(futures):
                future.result()

https://www.gamerbraves.com/category/news/


100%|█████████████████████████████████████████| 152/152 [33:55<00:00, 13.39s/it]


https://www.gamerbraves.com/category/reviews/


100%|███████████████████████████████████████████| 11/11 [01:49<00:00,  9.93s/it]


https://www.gamerbraves.com/category/interview/


100%|█████████████████████████████████████████████| 4/4 [00:32<00:00,  8.18s/it]
